# Option Pricing Logic for all Yahoo Finance Data

Make sure to create the required files first in other notebook(s) before evaluating them here

In [2]:
import os
# set working directory to MarketGenerators folder
# if you are working on LRZ servers, create the folder "MarketGenerators" and then specify something like
path = "/dss/dsshome1/02/YOUR_LRZ_USER_NAME/MarketGenerators"
os.chdir(path)

In [1]:
!pip install matplotlib

In [3]:
import numpy as np

from src.evaluate import metrics as m
from src.visualization.plot_option_results import OptionPricingVisualization 
import src.helper.utils as utils

In [ ]:
# Define the base directory
base_dir = "numerical_results"
# Define the folders of interest
model_folders = ["YFinance"]
folders_to_include = [
    "n-in=0Y" # For YFinance data, we set n_in=0Y always
]
relevant_n_Days = [5, 10, 21, 252]
# Define the target subfolders (CVAE only available if Buehler code was run)
target_subfolders = [
    "CWGAN", "GMMN", "RCGAN", "SigCWGAN", "TimeGAN", 
    "CVAE"
]
model_count = 0
# Parameters
S0 = 1.

In [ ]:
for nDays in relevant_n_Days:
    # Calculate Lookback option price every look_back_grid_size days
    look_back_grid_size = 1 if nDays <= 21 else 10
    T = nDays/252
    t = 0
    # Specify K_grid for output plots here
    K_grid = np.linspace(0.95, 1.1, 25) if nDays <= 5 else np.linspace(0.925, 1.1, 29) if nDays <= 10 else np.linspace(0.9125, 1.1, 31) if nDays <= 21 else np.linspace(0.8, 1.4, 49)
    # Initialize epmty last specification
    last_spec=""

    # Loop through each model folder
    for model_folder in model_folders:
        print(f"Start evaluating all {model_folder}-based models for {nDays} days maturity...")
        model_path = os.path.join(base_dir, model_folder)

        # Traverse the directory tree and find relevant files
        for root, dirs, files in os.walk(model_path):
            gen_model = os.path.basename(root)

            if gen_model in target_subfolders:
                nYearsFolderName = root.split("/")[3] 
                if nYearsFolderName not in folders_to_include:
                    continue
                # only price options for seed=42 (others are retrained models)
                if root.split("/")[4] != "seed=42":
                    continue

                # Read the relevant npy files
                generated_file = os.path.join(root, "generated_returns_rescaled.npy")
                input_file = os.path.join(root, "input_returns_unscaled.npy")
                if os.path.exists(generated_file) and (os.path.exists(input_file) or gen_model =="CVAE"):
                    if gen_model == "CVAE":
                        # no input data saved for CVAE model currently
                        input_file = os.path.join(root[:-4]+"SigCWGAN", "input_returns_unscaled.npy")
 
                    input_prices_df, generated_prices_df, _ = utils.load_input_and_generated_returns(
                        input_file, generated_file, nDays, T
                    )
                    input_model, model_spec = root.split("/")[1:3] 

                    # First folder => New evaluation
                    if model_count == 0:
                        new_input = True
                        closePlots = False
                        # Reset output string
                        summary_output = ""
                        model_desc = "/".join(root.split("/")[1:5])
                        print(f"   Evaluating model {model_desc}...")
                        
                        if model_spec != last_spec:
                            # Recalculate important option data as new input model is used
                            print("      New specification")
                            last_spec = model_spec
                            european_engine, asian_engine, lookback_engine = utils.initialize_all_option_engines(
                                input_prices_df, generated_prices_df, T, t=t, S0=S0, input_is_real_data=True
                            )

                    input_spec = "-".join(root.split("/")[1:5]).replace(".", ",") + f"_nDays={nDays}"

                    with utils.Capturing(summary_output) as summary_output:
                        # Get respective path metrics and save drift for MC
                        print(gen_model + " " + input_spec)
                        european_engine.r = m.print_basic_non_gbm_metrics( 
                            n_periods=T,
                            annualization_factor = 252, 
                            ground_paths_df = input_prices_df, 
                            recovered_paths_df = generated_prices_df, 
                            approx_df=input_prices_df,
                            return_threshold = 0.03
                        )
                        asian_engine.r = european_engine.r
                        lookback_engine.r = european_engine.r

                    print(f"         Calculating option prices for {gen_model} paths")
                    # calculate all values (option prices & deviations) for different strike prices (K or T values)
                    european_engine.gen_paths_df=generated_prices_df
                    european_engine.ground_paths_df=input_prices_df
                    european_engine.calc_all_K(K_grid, recalculate_input=new_input)
                    asian_engine.gen_paths_df=generated_prices_df
                    asian_engine.ground_paths_df=input_prices_df
                    asian_engine.calc_all_K(K_grid, recalculate_input=new_input)
                    lookback_engine.gen_paths_df=generated_prices_df
                    lookback_engine.ground_paths_df=input_prices_df
                    lookback_engine.calc_all_T(grid_size=look_back_grid_size, recalculate_input=new_input)
                    new_input = False
                    # Plot all types of plots
                    if model_count == 0:
                        # Initialize new plotter if new setup is present
                        european_plotter = OptionPricingVisualization(european_engine, file_name=input_spec, exact_label="Input ")
                        asian_plotter = OptionPricingVisualization(asian_engine, file_name=input_spec, exact_label="Input ")
                        lookback_plotter = OptionPricingVisualization(lookback_engine, file_name=input_spec, exact_label="Input ")

                    # Update pricing engine for respective model
                    european_plotter.pe = european_engine
                    asian_plotter.pe = asian_engine
                    lookback_plotter.pe = lookback_engine
                    model_count += 1
                    
                    # Save all results to files if setup is done
                    if model_count == len(target_subfolders):
                        # Specify file location
                        relevant_dir = "/".join(root.split("/")[:-1])
                        print(f"            Writing summary file at {relevant_dir}.")
                        with open(f"{relevant_dir}/basic_metrics_nDays={nDays}.txt", "w") as text_file:
                            text_file.write("\n".join(summary_output))
                        closePlots = True
                        # Reset model counter
                        model_count = 0

                    # Plot option results (folder specified in function)
                    european_plotter.plot_option_prices(close=closePlots, label=gen_model)
                    european_plotter.plot_option_price_deviation(close=closePlots, label=gen_model)
                    european_plotter.plot_option_price_deviation_relative(close=closePlots, label=gen_model, zoom_ylimits=(0,55))

                    asian_plotter.plot_option_prices(close=closePlots, label=gen_model)
                    asian_plotter.plot_option_price_deviation(close=closePlots, label=gen_model)
                    asian_plotter.plot_option_price_deviation_relative(close=closePlots, label=gen_model, zoom_ylimits=(0,55))

                    lookback_plotter.plot_option_prices(close=closePlots, label=gen_model)
                    lookback_plotter.plot_option_price_deviation(close=closePlots, label=gen_model)
                    lookback_plotter.plot_option_price_deviation_relative(close=closePlots, label=gen_model, zoom_ylimits=(0,55))
                
                else:
                    print("No file found yet.")

print("Done.")